# Building Agents That Use Code

This notebook is part of the [Hugging Face Agents Course](https://www.hf.co/learn/agents-course), a free Course from beginner to expert, where you learn to build Agents.

![Agents course share](https://huggingface.co/datasets/agents-course/course-images/resolve/main/en/communication/share.png)

## Let's install the dependencies and login to our HF account to access the Inference API

If you haven't installed `smolagents` yet, you can do so by running the following command:

Let's also login to the Hugging Face Hub to have access to the Inference API.

In [3]:
from dotenv import load_dotenv
from smolagents import LiteLLMModel
load_dotenv()

MODEL_ID = "ollama_chat/qwen2.5-coder:7b"

model = LiteLLMModel(
    model_id=MODEL_ID,  # Or try other Ollama-supported models
    api_base="http://127.0.0.1:11434",  # Default Ollama local server
    num_ctx=8192
)

## The `@tool` Decorator  

### Generating a tool that retrieves the highest-rated catering

Let's imagine that Alfred has already decided on the menu for the party, but now he needs help preparing food for such a large number of guests. To do so, he would like to hire a catering service and needs to identify the highest-rated options available. Alfred can leverage a tool to search for the best catering services in his area.

Below is an example of how Alfred can use the `@tool` decorator to make this happen:

In [ ]:
from smolagents import CodeAgent, tool

# Let's pretend we have a function that fetches the highest-rated catering services.
@tool
def catering_service_tool(query: str) -> str:
    """
    This tool returns the highest-rated catering service in Gotham City.

    Args:
        query: A search term for finding catering services.
    """
    # Example list of catering services and their ratings
    services = {
        "Gotham Catering Co.": 4.9,
        "Wayne Manor Catering": 4.8,
        "Gotham City Events": 4.7,
    }

    # Find the highest rated catering service (simulating search query filtering)
    best_service = max(services, key=services.get)

    return best_service


agent = CodeAgent(tools=[catering_service_tool], model=model)

# Run the agent to find the best catering service
result = agent.run(
    "Can you give me the name of the highest-rated catering service in Gotham City?"
)

print(result)

## Defining a Tool as a Python Class  

### Generating a tool to generate ideas about the superhero-themed party

Alfred's party at the mansion is a **superhero-themed event**, but he needs some creative ideas to make it truly special. As a fantastic host, he wants to surprise the guests with a unique theme.

To do this, he can use an agent that generates superhero-themed party ideas based on a given category. This way, Alfred can find the perfect party theme to wow his guests.

In [ ]:
from smolagents import Tool, CodeAgent

class SuperheroPartyThemeTool(Tool):
    name = "superhero_party_theme_generator"
    description = """
    This tool suggests creative superhero-themed party ideas based on a category.
    It returns a unique party theme idea."""

    inputs = {
        "category": {
            "type": "string",
            "description": "The type of superhero party (e.g., 'classic heroes', 'villain masquerade', 'futuristic Gotham').",
        }
    }

    output_type = "string"

    def forward(self, category: str):
        themes = {
            "classic heroes": "Justice League Gala: Guests come dressed as their favorite DC heroes with themed cocktails like 'The Kryptonite Punch'.",
            "villain masquerade": "Gotham Rogues' Ball: A mysterious masquerade where guests dress as classic Batman villains.",
            "futuristic Gotham": "Neo-Gotham Night: A cyberpunk-style party inspired by Batman Beyond, with neon decorations and futuristic gadgets."
        }

        return themes.get(category.lower(), "Themed party idea not found. Try 'classic heroes', 'villain masquerade', or 'futuristic Gotham'.")

# Instantiate the tool
party_theme_tool = SuperheroPartyThemeTool()
agent = CodeAgent(tools=[party_theme_tool], model=model)

# Run the agent to generate a party theme idea
result = agent.run(
    "What would be a good superhero party idea for a 'villain masquerade' theme?"
)

print(result)

## Sharing a Tool to the Hub

Sharing your custom tool with the community is easy! Simply upload it to your Hugging Face account using the `push_to_hub()` method.

For instance, Alfred can share his `catering_service_tool` to help others find the best catering services in Gotham. Here's how to do it:

In [ ]:
# party_theme_tool.push_to_hub("{your_username}/catering_service_tool", token="<YOUR_HUGGINGFACEHUB_API_TOKEN>")

## Importing a Tool from the Hub

You can easily import tools created by other users using the `load_tool()` function. For example, Alfred might want to generate a promotional image for the party using AI. Instead of building a tool from scratch, he can leverage a predefined one from the community:

In [ ]:
from smolagents import load_tool, CodeAgent
from PIL import Image

image_generation_tool = Tool.from_space(
    "black-forest-labs/FLUX.1-schnell",
    name="image_generator",
    description="Generate an image from a prompt"
)

agent = CodeAgent(
    tools=[image_generation_tool],
    model=model
)

image_file = agent.run("Generate an image of a luxurious superhero-themed party at Wayne Manor with made-up superheros.")
image = Image.open(image_file)
image.show()

## Importing a Hugging Face Space as a Tool

You can also import a HF Space as a tool using `Tool.from_space()`. This opens up possibilities for integrating with thousands of spaces from the community for tasks from image generation to data analysis.

The tool will connect with the spaces Gradio backend using the `gradio_client`, so make sure to install it via `pip` if you don't have it already. For the party, Alfred can also use a HF Space directly for the generation of the previous annoucement AI-generated image. Let's build it!

In [ ]:
!pip install gradio_client

In [ ]:
from smolagents import load_tool, CodeAgent
from PIL import Image

image_generation_tool = Tool.from_space(
    "black-forest-labs/FLUX.1-schnell",
    name="image_generator",
    description="Generate an image from a prompt"
)

# model = InferenceClientModel("Qwen/Qwen2.5-Coder-32B-Instruct")

agent = CodeAgent(tools=[image_generation_tool], model=model)

another_image_file = agent.run(
    "Improve this prompt, then generate an image of it.",
    additional_args={'user_prompt': 'A grand superhero-themed party at Wayne Manor, with Alfred overseeing a luxurious gala'}
)
image = Image.open(another_image_file)
image.show()

## Importing a LangChain Tool

These tools need a [SerpApi API Key](https://serpapi.com/).

You can easily load LangChain tools using the `Tool.from_langchain()` method. Alfred, ever the perfectionist, is preparing for a spectacular superhero night at Wayne Manor while the Waynes are away. To make sure every detail exceeds expectations, he taps into LangChain tools to find top-tier entertainment ideas.

By using `Tool.from_langchain()`, Alfred effortlessly adds advanced search functionalities to his smolagent, enabling him to discover exclusive party ideas and services with just a few commands.

Here's how he does it:

In [4]:
from langchain.agents import load_tools
from smolagents import CodeAgent, Tool

serp_api_tool = Tool.from_langchain(load_tools(["serpapi"])[0])

agent = CodeAgent(tools=[serp_api_tool], model=model)

agent.run("Search for luxury entertainment ideas for a superhero-themed event, such as live performances and interactive experiences.")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Search for luxury entertainment ideas for a superhero-themed event, such as live performances and interactive   │
│ experiences.                                                                                                    │
│                                                                                                                 │
╰─ LiteLLMModel - ollama_chat/qwen2.5-coder:7b ───────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  # Search for luxury entertainment options related to superhero themes                                            
  luxury_entertainment_options = search("luxury entertainment options for superhero-themed events")                
  print(f"Luxury Entertainment Options: {luxury_entertainment_options}")                                           
                                                                                                                   
  # Search for packages and services for superhero-themed events                                                   
  superhero_event_packages = search("packages for superhero-themed events")                                        
  print(f"Superhero Event Packages: {superhero_event_packages}")                                                   
                                                                                                                   
  # Generate suggestions based on the collected information                                                        
  suggestions = []                                                                                                 
  if "live performances" in luxury_entertainment_options:                                                          
      suggestions.append("Live performances such as live orchestras, dance shows, or magic shows.")                
  if "interactive experiences" in luxury_entertainment_options:                                                    
      suggestions.append("Interactive experiences like escape rooms, superhero training workshops, or interactive  
  games.")                                                                                                         
                                                                                                                   
  # If any relevant packages were found, add them to the suggestions                                               
  for package in superhero_event_packages:                                                                         
      if "luxury" in package.lower():                                                                              
          suggestions.append(package)                                                                              
                                                                                                                   
  final_answer(suggestions)                                                                                        
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
Luxury Entertainment Options: ['Our comprehensive package includes a wide array of exciting features such as 
Superhero obstacle course, Sports and the option to dress up in superhero costumes, ...', 'When in doubt, throw a 
superhero party - this party theme will save the day! Be the hero of your own story, and create the ultimate 
superhero themed event.', "Unleash your team's inner heroes with the ultimate superhero-themed event experience! 
From thrilling rides and immersive attractions to all-inclusive ...", "Here's a look at different ideas for adding 
cartoon-like visuals and references to caped crusaders at your next affair.", 'Super Fun Superhero Event Theme ; 
three lamps with comic covers on them. Lamp In A Box Vintage Comic Shades · Superhero Living Room Decor ; a man and
woman ...', 'Classic Superhero Costume Party\u200b\u200b Decorations can include comic book-style banners, 
life-sized cut outs of popular heroes, and a colourful backdrop that screams ...', "We've created this guide to 
take you through some of the best ideas out there for decorating and styling a superhero theme in order to create a
thrilling ...", 'This ultimate Marvel Party Ideas will unveil secrets to creating an immersive experience, from 
heroic decorations to action-packed entertainment.', 'Decorating the Venue · Transforming Spaces into Heroic Scenes
· Villainous Lair Inspirations · Super Props and Centerpieces · DIY Decoration Ideas.', "From pre-designed 
superhero experiences to fully bespoke creations tailored to your favourite characters, we're here to help you 
throw a party ..."]
Superhero Event Packages: ['Experience superhero training, your own official cape and mask, an honorary superhero 
certificate, and MORE with our SUPERHERO Packages!', 'Voted the #1 Superhero Party Company in Toronto, we 
specialize in Superhero and Star Warrior Birthday Parties, corporate appearances, character meet and greets, ...', 
'Bring the thrill of adventure to your event with our superhero packages! Featuring heroic characters and 
superhero-themed games, we create unforgettable ...', 'Our comprehensive package includes a wide array of exciting 
features such as Superhero obstacle course, Sports and the option to dress up in superhero costumes, ...', '\u200d♂️
SUPERHERO ADVENTURE PACKAGE · ONE HOUR. 1 Character: $245. 2 Characters: $345. 3 Characters: $470. 4 Characters: 
$595 · TWO HOURS. 1 Character: $345. 2 ...', 'Our super heroes birthday party package includes thrilling games, 
heroic activities, and photo opportunities that will leave everyone feeling like true ...', 'We offer several 
superhero party characters & affordable party packages.Black Panther,Superman,Wonder Woman,Iron Man party.', 'Find 
superhero party packages at the lowest price guaranteed. Buy today & save plus get free shipping offers on all 
theme packs at OrientalTrading.com!', "This all-inclusive package includes superhero costumes, masks, and 
decorations to transform your party into an epic superhero adventure. Whether you're hosting ...", 'This 
power-packed 1 hour party package, with the superhero of your choice, includes: *Superhero Training. *Superhero 
workouts. *Caping of the birthday child.']

Out - Final answer: []

[Step 1: Duration 29.71 seconds| Input tokens: 2,082 | Output tokens: 268]

[]

With this setup, Alfred can quickly discover luxurious entertainment options, ensuring Gotham's elite guests have an unforgettable experience. This tool helps him curate the perfect superhero-themed event for Wayne Manor! 🎉